**Imports**

In [2]:
import warnings
from bert import *
from utils import *
from transformers import BertTokenizer

torch.manual_seed(0)
np.random.seed(0)
warnings.filterwarnings("ignore")

model_name, maxlen = 'prajjwal1/bert-tiny', 128
tokenizer = BertTokenizer.from_pretrained(model_name)
sts_dataset = load_sts_dataset('data/stsbenchmark.tsv.gz')
tokenized_test = tokenize_sentence_pair_dataset(sts_dataset['test'], tokenizer)
sts_testloader = get_dataloader(tokenized_test, batch_size=1)

# Evaluate baseline TinyBERT model on STS benchmark

In [2]:
bert_config = {"hidden_size": 128, "num_attention_heads": 2, "num_hidden_layers": 2, "intermediate_size": 512, "vocab_size": 30522}
bert_path = 'weights/bert_tiny.bin'
bert = Bert(bert_config).load_model(bert_path)
print('Parameters:', sum(p.numel() for p in bert.parameters() if p.requires_grad))

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
results_from_pretrained = eval_loop(bert, sts_testloader, device)
torch.save(bert.state_dict(), 'weights/baseline_bert.pth')
print(f'\nPearson correlation: {results_from_pretrained[0]:.2f}\nSpearman correlation: {results_from_pretrained[1]:.2f}')

Parameters: 4385920

Pearson correlation: 0.43
Spearman correlation: 0.42


# Train TinyBERT with NLI Classifier and evaluate

In [3]:
class Classifier(nn.Module):
    def __init__(self,
                 bert_config,
                 bert_path: str):
        """
        Initialize a BERT-based classifier model.

        Args:
            bert_config: BERT model configuration.
            bert_path (str): Path to a pre-trained BERT model checkpoint.

        Attributes:
            bert (Bert): Pre-trained BERT model.

        """
        super(Classifier, self).__init__()
        self.bert = Bert(bert_config).load_model(bert_path)
        self.classification_head = nn.Linear(128*4,2)
        self.criterion = nn.CrossEntropyLoss()

    def forward(self,
                sentence_1_ids: torch.Tensor,
                sentence_1_mask: torch.Tensor,
                sentence_2_ids: torch.Tensor,
                sentence_2_mask: torch.Tensor,
                labels: torch.Tensor) -> torch.Tensor:
        
        embeddings_1 = self.bert(input_ids=sentence_1_ids, attention_mask=sentence_1_mask)[1]
        embeddings_2 = self.bert(input_ids=sentence_2_ids, attention_mask=sentence_2_mask)[1]
        return self.loss(embeddings_1, embeddings_2, labels)

    def loss(self, a: torch.Tensor, b: torch.Tensor, labels: torch.Tensor) -> torch.Tensor:
        logits = self.classification_head(torch.cat((a, b, torch.abs(a-b), a*b), dim=-1))
        preds = nn.functional.softmax(logits, dim=-1)
        return self.criterion(preds, labels)

model = train_and_evaluate(Classifier, tokenizer, sts_testloader, num_epochs=30, batch_size=256, maxlen=maxlen)
torch.save(model.bert.state_dict(), 'weights/classifier_bert.pth')

Train: 645798
Validation: 13209
4386946

Pearson correlation: 0.47
Spearman correlation: 0.47
Epoch [1/30] - Train Loss: 0.5929, Validation Loss: 0.5566

Pearson correlation: 0.50
Spearman correlation: 0.50
Epoch [2/30] - Train Loss: 0.5384, Validation Loss: 0.5289

Pearson correlation: 0.51
Spearman correlation: 0.52
Epoch [3/30] - Train Loss: 0.5153, Validation Loss: 0.5156

Pearson correlation: 0.53
Spearman correlation: 0.55
Epoch [4/30] - Train Loss: 0.5005, Validation Loss: 0.5076

Pearson correlation: 0.54
Spearman correlation: 0.55
Epoch [5/30] - Train Loss: 0.4886, Validation Loss: 0.5012

Pearson correlation: 0.53
Spearman correlation: 0.55
Epoch [6/30] - Train Loss: 0.4790, Validation Loss: 0.5006

Pearson correlation: 0.54
Spearman correlation: 0.56
Epoch [7/30] - Train Loss: 0.4706, Validation Loss: 0.4982

Pearson correlation: 0.55
Spearman correlation: 0.56
Epoch [8/30] - Train Loss: 0.4636, Validation Loss: 0.4949

Pearson correlation: 0.54
Spearman correlation: 0.56
Ep

# Train TinyBERT with Contrastive Loss on all data and evaluate

In [8]:
class NTXent(nn.Module):
    def __init__(self,
                 bert_config,
                 bert_path: str):
        """
        Initialize a BERT-based classifier model.

        Args:
            bert_config: BERT model configuration.
            bert_path (str): Path to a pre-trained BERT model checkpoint.

        Attributes:
            bert (Bert): Pre-trained BERT model.

        """
        super(NTXent, self).__init__()
        self.bert = Bert(bert_config).load_model(bert_path)
        self.temperature = nn.Parameter(torch.tensor([0.01]))

    def forward(self,
                sentence_1_ids: torch.Tensor,
                sentence_1_mask: torch.Tensor,
                sentence_2_ids: torch.Tensor,
                sentence_2_mask: torch.Tensor,
                labels: torch.Tensor) -> torch.Tensor:
        
        embeddings_1 = self.bert(input_ids=sentence_1_ids, attention_mask=sentence_1_mask)[1]
        embeddings_2 = self.bert(input_ids=sentence_2_ids, attention_mask=sentence_2_mask)[1]
        return self.loss(embeddings_1, embeddings_2)

    def loss(self, a: torch.Tensor, b: torch.Tensor) -> torch.Tensor:
        features = torch.cat([a, b], dim=0)
        normalized_features = F.normalize(features, p=2, dim=1)
        similarity_matrix = torch.mm(normalized_features, normalized_features.T)
        logits = similarity_matrix / self.temperature
        batch_size = a.size(0)
        mask = torch.eye(batch_size, dtype=torch.bool, device=a.device)
        labels = torch.cat([torch.arange(batch_size, device=a.device), torch.arange(batch_size, device=a.device)])
        logits = logits.masked_select(~torch.block_diag(mask, mask)).view(2 * batch_size, -1)
        loss = F.cross_entropy(logits, labels)
        return loss
    
model = train_and_evaluate(NTXent, tokenizer, sts_testloader, num_epochs=5, batch_size=256, maxlen=maxlen)
torch.save(model.bert.state_dict(), 'weights/ntxent_bert.pth')

Train: 645798
Validation: 13209
4385921

Pearson correlation: 0.59
Spearman correlation: 0.62
Epoch [1/5] - Train Loss: 4.8601, Validation Loss: 4.6939

Pearson correlation: 0.61
Spearman correlation: 0.63
Epoch [2/5] - Train Loss: 4.5413, Validation Loss: 4.5590

Pearson correlation: 0.60
Spearman correlation: 0.62
Epoch [3/5] - Train Loss: 4.4258, Validation Loss: 4.5711

Pearson correlation: 0.59
Spearman correlation: 0.61
Epoch [4/5] - Train Loss: 4.3490, Validation Loss: 4.5083

Pearson correlation: 0.57
Spearman correlation: 0.60
Epoch [5/5] - Train Loss: 4.2917, Validation Loss: 4.6276


# Train TinyBERT with Triplet Loss on all data and evaluate

In [5]:
class Triplet(nn.Module):
    def __init__(self,
                 bert_config,
                 bert_path: str):
        """
        Initialize a BERT-based classifier model.

        Args:
            bert_config: BERT model configuration.
            bert_path (str): Path to a pre-trained BERT model checkpoint.

        Attributes:
            bert (Bert): Pre-trained BERT model.

        """
        super(Triplet, self).__init__()
        self.bert = Bert(bert_config).load_model(bert_path)
        self.loss_fn = nn.TripletMarginLoss(margin=1.0)

    def forward(self,
                sentence_1_ids: torch.Tensor,
                sentence_1_mask: torch.Tensor,
                sentence_2_ids: torch.Tensor,
                sentence_2_mask: torch.Tensor,
                labels: torch.Tensor) -> torch.Tensor:
        
        embeddings_1 = self.bert(input_ids=sentence_1_ids, attention_mask=sentence_1_mask)[1]
        embeddings_2 = self.bert(input_ids=sentence_2_ids, attention_mask=sentence_2_mask)[1]
        return self.loss(embeddings_1, embeddings_2, labels)

    def loss(self, anchors, positives, labels):
        distance_matrix = torch.cdist(anchors, positives, p=2)
        labels_expanded = labels.unsqueeze(1)
        negative_mask = labels_expanded != labels_expanded.t()
        max_distance = distance_matrix.max().detach()
        distance_matrix_masked = torch.where(negative_mask, distance_matrix, max_distance)
        hardest_negatives = positives[distance_matrix_masked.argmin(dim=1)]
        loss = self.loss_fn(anchors, positives, hardest_negatives)
        return loss

model = train_and_evaluate(Triplet, tokenizer, sts_testloader, num_epochs=30, batch_size=256, maxlen=maxlen)
torch.save(model.bert.state_dict(), 'weights/triplet_bert.pth')

Train: 645798
Validation: 13209
4385920

Pearson correlation: 0.59
Spearman correlation: 0.62
Epoch [1/30] - Train Loss: 0.7692, Validation Loss: 0.5963

Pearson correlation: 0.62
Spearman correlation: 0.64
Epoch [2/30] - Train Loss: 0.6293, Validation Loss: 0.5580

Pearson correlation: 0.65
Spearman correlation: 0.66
Epoch [3/30] - Train Loss: 0.5811, Validation Loss: 0.5309

Pearson correlation: 0.64
Spearman correlation: 0.65
Epoch [4/30] - Train Loss: 0.5500, Validation Loss: 0.5229

Pearson correlation: 0.66
Spearman correlation: 0.66
Epoch [5/30] - Train Loss: 0.5258, Validation Loss: 0.5044

Pearson correlation: 0.66
Spearman correlation: 0.66
Epoch [6/30] - Train Loss: 0.5078, Validation Loss: 0.5006

Pearson correlation: 0.66
Spearman correlation: 0.66
Epoch [7/30] - Train Loss: 0.4908, Validation Loss: 0.4926

Pearson correlation: 0.67
Spearman correlation: 0.67
Epoch [8/30] - Train Loss: 0.4782, Validation Loss: 0.4869

Pearson correlation: 0.66
Spearman correlation: 0.66
Ep

# Train TinyBERT with Warping Loss on entailment data alone

In [2]:
class Warping(nn.Module):
    def __init__(self,
                 bert_config,
                 bert_path: str):
        """
        Initialize a BERT-based classifier model.

        Args:
            bert_config: BERT model configuration.
            bert_path (str): Path to a pre-trained BERT model checkpoint.

        Attributes:
            bert (Bert): Pre-trained BERT model.

        """
        super(Warping, self).__init__()
        self.bert = Bert(bert_config).load_model(bert_path)
        self.temperature = nn.Parameter(torch.tensor([0.01]))
        self.projection = nn.Linear(128*4,256)
        self.mu = 0.3
        self.std = 0.2
        self.n_warps = 100

    def forward(self,
                sentence_1_ids: torch.Tensor,
                sentence_1_mask: torch.Tensor,
                sentence_2_ids: torch.Tensor,
                sentence_2_mask: torch.Tensor,
                labels: torch.Tensor) -> torch.Tensor:
        """
        Forward pass of the BERT-based classifier.

        Args:
            sentence_1_ids (torch.Tensor): Input tensor for sentence 1 token IDs.
            sentence_1_mask (torch.Tensor): Input tensor for sentence 1 attention mask.
            sentence_2_ids (torch.Tensor): Input tensor for sentence 2 token IDs.
            sentence_2_mask (torch.Tensor): Input tensor for sentence 2 attention mask.
            labels (torch.Tensor): Ground truth class labels.

        Returns:
            torch.Tensor: Loss value.

        """
        embeddings_1 = self.bert(input_ids=sentence_1_ids, attention_mask=sentence_1_mask, training=True)[1]
        embeddings_2 = self.bert(input_ids=sentence_2_ids, attention_mask=sentence_2_mask, training=True)[1]
        return self.loss(embeddings_1, embeddings_2)

    def warp(self, embeddings: torch.Tensor) -> torch.Tensor:
        shape = embeddings.shape
        scales = torch.normal(mean=self.mu, std=self.std, size=shape) + torch.ones(shape)
        embeddings = torch.mul(embeddings, scales.to(embeddings.device))
        return embeddings

    def loss(self, a: torch.Tensor, b: torch.Tensor) -> torch.Tensor:
        a = self.warp(F.normalize(a, p=2, dim=1))
        b = self.warp(F.normalize(b, p=2, dim=1))
        c = torch.cat((a,b,torch.abs(a-b)), dim=-1)
        similarity_matrix = (a @ b.T) / self.temperature 
        labels = torch.arange(similarity_matrix.shape[0]).to(similarity_matrix.device)
        loss_a = F.cross_entropy(similarity_matrix, labels)
        loss_b = F.cross_entropy(similarity_matrix.T, labels)
        return (loss_a + loss_b)/2 
    
model = train_and_evaluate(Warping, tokenizer, sts_testloader, num_epochs=30, batch_size=256, maxlen=maxlen)
torch.save(model.bert.state_dict(), 'weights/warping_bert.pth')

Train: 322742
Validation: 6452
4517249

Pearson correlation: 0.69
Spearman correlation: 0.68
Epoch [1/30] - Train Loss: 2.4485, Validation Loss: 1.4134

Pearson correlation: 0.70
Spearman correlation: 0.70
Epoch [2/30] - Train Loss: 1.4441, Validation Loss: 1.1642

Pearson correlation: 0.71
Spearman correlation: 0.70
Epoch [3/30] - Train Loss: 1.1857, Validation Loss: 1.0127

Pearson correlation: 0.71
Spearman correlation: 0.71
Epoch [4/30] - Train Loss: 1.0374, Validation Loss: 0.9618

Pearson correlation: 0.72
Spearman correlation: 0.71
Epoch [5/30] - Train Loss: 0.9365, Validation Loss: 0.8999

Pearson correlation: 0.72
Spearman correlation: 0.71
Epoch [6/30] - Train Loss: 0.8625, Validation Loss: 0.8598

Pearson correlation: 0.72
Spearman correlation: 0.71
Epoch [7/30] - Train Loss: 0.8006, Validation Loss: 0.8459

Pearson correlation: 0.72
Spearman correlation: 0.71
Epoch [8/30] - Train Loss: 0.7528, Validation Loss: 0.8360

Pearson correlation: 0.72
Spearman correlation: 0.71
Epo

# Experiments

In [92]:
class Warping(nn.Module):
    def __init__(self,
                 bert_config,
                 bert_path: str):
        """
        Initialize a BERT-based classifier model.

        Args:
            bert_config: BERT model configuration.
            bert_path (str): Path to a pre-trained BERT model checkpoint.

        Attributes:
            bert (Bert): Pre-trained BERT model.

        """
        super(Warping, self).__init__()
        self.bert = Bert(bert_config).load_model(bert_path)
        self.temperature = nn.Parameter(torch.tensor([0.01]))
        self.projection = nn.Linear(128*4,256)
        self.mu = 0.3
        self.std = 0.2
        self.n_warps = 100

    def forward(self,
                sentence_1_ids: torch.Tensor,
                sentence_1_mask: torch.Tensor,
                sentence_2_ids: torch.Tensor,
                sentence_2_mask: torch.Tensor,
                labels: torch.Tensor) -> torch.Tensor:
        """
        Forward pass of the BERT-based classifier.

        Args:
            sentence_1_ids (torch.Tensor): Input tensor for sentence 1 token IDs.
            sentence_1_mask (torch.Tensor): Input tensor for sentence 1 attention mask.
            sentence_2_ids (torch.Tensor): Input tensor for sentence 2 token IDs.
            sentence_2_mask (torch.Tensor): Input tensor for sentence 2 attention mask.
            labels (torch.Tensor): Ground truth class labels.

        Returns:
            torch.Tensor: Loss value.

        """
        embeddings_1 = self.bert(input_ids=sentence_1_ids, attention_mask=sentence_1_mask, training=True)[1]
        embeddings_2 = self.bert(input_ids=sentence_2_ids, attention_mask=sentence_2_mask, training=True)[1]
        return self.loss(embeddings_1, embeddings_2)

    def warp(self, embeddings: torch.Tensor) -> torch.Tensor:
        scales = torch.normal(mean=0.0, std=0.2, size=embeddings.shape) + 1
        embeddings = embeddings * scales.to(embeddings.device)
        return F.normalize(embeddings, p=2, dim=1)

    def loss(self, a: torch.Tensor, b: torch.Tensor) -> torch.Tensor:
        a, b = self.warp(a), self.warp(b)
        similarity_matrix = (a @ b.T) / self.temperature 
        labels = torch.arange(end=a.shape[0], device=a.device)
        return F.cross_entropy(similarity_matrix.T, labels)
    
model = train_and_evaluate(Warping, tokenizer, sts_testloader, num_epochs=30, batch_size=256, maxlen=maxlen)
torch.save(model.bert.state_dict(), 'weights/experimental.pth')

Train: 3622
Validation: 319
4517249

Pearson correlation: 0.50
Spearman correlation: 0.49
Epoch [1/30] - Train Loss: 4.4451, Validation Loss: 3.4558

Pearson correlation: 0.56
Spearman correlation: 0.54
Epoch [2/30] - Train Loss: 3.8975, Validation Loss: 3.1812

Pearson correlation: 0.58
Spearman correlation: 0.56
Epoch [3/30] - Train Loss: 3.5175, Validation Loss: 3.2296

Pearson correlation: 0.61
Spearman correlation: 0.58
Epoch [4/30] - Train Loss: 3.2363, Validation Loss: 2.8587

Pearson correlation: 0.62
Spearman correlation: 0.59
Epoch [5/30] - Train Loss: 3.0322, Validation Loss: 2.7317

Pearson correlation: 0.62
Spearman correlation: 0.60
Epoch [6/30] - Train Loss: 2.8170, Validation Loss: 2.7975

Pearson correlation: 0.63
Spearman correlation: 0.61
Epoch [7/30] - Train Loss: 2.6958, Validation Loss: 2.6720

Pearson correlation: 0.63
Spearman correlation: 0.61
Epoch [8/30] - Train Loss: 2.5662, Validation Loss: 2.6377

Pearson correlation: 0.64
Spearman correlation: 0.61
Epoch 

In [9]:
class Warping(nn.Module):
    def __init__(self,
                 bert_config,
                 bert_path: str):
        """
        Initialize a BERT-based classifier model.

        Args:
            bert_config: BERT model configuration.
            bert_path (str): Path to a pre-trained BERT model checkpoint.

        Attributes:
            bert (Bert): Pre-trained BERT model.

        """
        super(Warping, self).__init__()
        self.bert = Bert(bert_config).load_model(bert_path)
        self.temperature = nn.Parameter(torch.tensor([0.01]))
        self.projection = nn.Linear(128*4,256)
        self.mu = 0.3
        self.std = 0.2
        self.n_warps = 100

    def forward(self,
                sentence_1_ids: torch.Tensor,
                sentence_1_mask: torch.Tensor,
                sentence_2_ids: torch.Tensor,
                sentence_2_mask: torch.Tensor,
                labels: torch.Tensor) -> torch.Tensor:
        """
        Forward pass of the BERT-based classifier.

        Args:
            sentence_1_ids (torch.Tensor): Input tensor for sentence 1 token IDs.
            sentence_1_mask (torch.Tensor): Input tensor for sentence 1 attention mask.
            sentence_2_ids (torch.Tensor): Input tensor for sentence 2 token IDs.
            sentence_2_mask (torch.Tensor): Input tensor for sentence 2 attention mask.
            labels (torch.Tensor): Ground truth class labels.

        Returns:
            torch.Tensor: Loss value.

        """
        embeddings_1 = self.bert(input_ids=sentence_1_ids, attention_mask=sentence_1_mask, training=True)[1]
        embeddings_2 = self.bert(input_ids=sentence_2_ids, attention_mask=sentence_2_mask, training=True)[1]
        return self.loss(embeddings_1, embeddings_2)

    def loss(self, a: torch.Tensor, b: torch.Tensor) -> torch.Tensor:
        a = F.normalize(a, p=2, dim=1)
        b = F.normalize(b, p=2, dim=1)
        similarity_matrix = (b @ a.T) / self.temperature 
        labels = torch.arange(end=a.shape[0], device=a.device) 
        return F.cross_entropy(similarity_matrix, labels)
    
model = train_and_evaluate(Warping, tokenizer, sts_testloader, num_epochs=30, batch_size=256, maxlen=maxlen)
torch.save(model.bert.state_dict(), 'weights/experimental.pth')

A person on a horse jumps over a broken down airplane.
A person is outdoors, on a horse.

Children smiling and waving at camera
There are children present

A boy is jumping on skateboard in the middle of a red bridge.
The boy does a skateboarding trick.

Two blond women are hugging one another.
There are women showing affection.

A few people in a restaurant setting, one of them is drinking orange juice.
The diners are at a restaurant.

An older man is drinking orange juice at a restaurant.
A man is drinking juice.

A man with blond-hair, and a brown shirt drinking out of a public water fountain.
A blond man drinking water from a fountain.

Two women who just had lunch hugging and saying goodbye.
There are two woman in this picture.

Two women, holding food carryout containers, hug.
Two women hug each other.

A Little League team tries to catch a runner sliding into a base in an afternoon game.
A team is trying to tag a runner out.

Train: 3622
Validation: 319
4517249

Pearson correlat

In [4]:
def loss(a: torch.Tensor, b: torch.Tensor) -> torch.Tensor:
    a = F.normalize(a, p=2, dim=1)
    b = F.normalize(b, p=2, dim=1)
    similarity_matrix = (a @ b.T) / 0.01
    
    print(similarity_matrix)
    labels = torch.arange(similarity_matrix.shape[0])
    print(labels)
    print(similarity_matrix.T)
    return F.cross_entropy(similarity_matrix.T, labels)

a = torch.randn(5, 10)
b = torch.randn(5, 10)
loss_value = loss(a, b)
print("Loss value:", loss_value.item())

tensor([[ 50.9167,  -2.1169,  39.5748, -24.6775,  88.2285],
        [  7.5358, -24.0080,  15.5092,  74.0901,  35.6088],
        [-12.0127,  26.0580,   2.0394,  -9.9870, -61.7342],
        [ 30.5556,  18.5826,  44.6844, -42.4859,  33.2288],
        [  1.2581,  50.2989,  18.0844,   9.9399, -19.9169]])
tensor([0, 1, 2, 3, 4])
tensor([[ 50.9167,   7.5358, -12.0127,  30.5556,   1.2581],
        [ -2.1169, -24.0080,  26.0580,  18.5826,  50.2989],
        [ 39.5748,  15.5092,   2.0394,  44.6844,  18.0844],
        [-24.6775,  74.0901,  -9.9870, -42.4859,   9.9399],
        [ 88.2285,  35.6088, -61.7342,  33.2288, -19.9169]])
Loss value: 68.3358383178711


In [5]:
loss_value = loss(b, a)

tensor([[ 50.9167,   7.5358, -12.0127,  30.5556,   1.2581],
        [ -2.1169, -24.0080,  26.0580,  18.5826,  50.2989],
        [ 39.5748,  15.5092,   2.0394,  44.6844,  18.0844],
        [-24.6775,  74.0901,  -9.9870, -42.4859,   9.9399],
        [ 88.2285,  35.6088, -61.7342,  33.2288, -19.9169]])
tensor([0, 1, 2, 3, 4])
tensor([[ 50.9167,  -2.1169,  39.5748, -24.6775,  88.2285],
        [  7.5358, -24.0080,  15.5092,  74.0901,  35.6088],
        [-12.0127,  26.0580,   2.0394,  -9.9870, -61.7342],
        [ 30.5556,  18.5826,  44.6844, -42.4859,  33.2288],
        [  1.2581,  50.2989,  18.0844,   9.9399, -19.9169]])


In [55]:
def loss2(a: torch.Tensor, b: torch.Tensor) -> torch.Tensor:
    features = torch.cat([a, b], dim=0)
    normalized_features = F.normalize(features, p=2, dim=1)
    similarity_matrix = torch.mm(normalized_features, normalized_features.T)
    logits = similarity_matrix / 0.01
    print(logits)
    batch_size = a.size(0)
    mask = torch.eye(batch_size, dtype=torch.bool, device=a.device)
    labels = torch.cat([torch.arange(batch_size, device=a.device), torch.arange(batch_size, device=a.device)])
    logits = logits.masked_select(~torch.block_diag(mask, mask)).view(2 * batch_size, -1)
    print(logits)
    loss = F.cross_entropy(logits, labels)
    return loss
loss2(a,a)

tensor([[100.0000, -22.5342,  68.4319,  29.2120, -45.7219, 100.0000, -22.5342,
          68.4319,  29.2120, -45.7219],
        [-22.5342, 100.0000,  47.3745, -34.0043,  59.5988, -22.5342, 100.0000,
          47.3745, -34.0043,  59.5988],
        [ 68.4319,  47.3745, 100.0000,  -7.5773,   7.4423,  68.4319,  47.3745,
         100.0000,  -7.5773,   7.4423],
        [ 29.2120, -34.0043,  -7.5773, 100.0000, -24.1734,  29.2120, -34.0043,
          -7.5773, 100.0000, -24.1734],
        [-45.7219,  59.5988,   7.4423, -24.1734, 100.0000, -45.7219,  59.5988,
           7.4423, -24.1734, 100.0000],
        [100.0000, -22.5342,  68.4319,  29.2120, -45.7219, 100.0000, -22.5342,
          68.4319,  29.2120, -45.7219],
        [-22.5342, 100.0000,  47.3745, -34.0043,  59.5988, -22.5342, 100.0000,
          47.3745, -34.0043,  59.5988],
        [ 68.4319,  47.3745, 100.0000,  -7.5773,   7.4423,  68.4319,  47.3745,
         100.0000,  -7.5773,   7.4423],
        [ 29.2120, -34.0043,  -7.5773, 100.0000,

tensor(55.2632)